In [23]:
!pip install rouge
!pip install rouge_metric
!pip install newspaper3k

You are using pip version 20.3.4, however version 21.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 20.3.4, however version 21.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 20.3.4, however version 21.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [24]:
!pip install feedparser==5.2.1
!pip install pandas

You are using pip version 20.3.4, however version 21.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 20.3.4, however version 21.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [25]:
import numpy as np
import pandas as pd
from keras.models import load_model
from word_embedding import embed_sentences
from dataload import loadTestData 
#from rouge import Rouge
import joblib
from rouge_metric import PyRouge
import nltk
from pprint import pprint
from newspaper import Article
from newspaper.article import ArticleDownloadState, ArticleException
from time import sleep
from nltk.tokenize import sent_tokenize

Using TensorFlow backend.
/home/duvvuri.s/anaconda3/envs/tensorflow/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/duvvuri.s/anaconda3/envs/tensorflow/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/duvvuri.s/anaconda3/envs/tensorflow/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])

In [3]:
def dummy_loadTestData():
    testing_data = [ [ np.array(["This sentence is important for doc0." ,
                                 "Such a sentence is irrelevent for doc 0."]), 
                       np.random.rand(2,5,300), 
                       np.array(["This sentence is important for doc0."]) ],
                     [ np.array(["Lol that sentence is awesome for do1." , 
                                 "No way, this is irrelevent"]), 
                       np.random.rand(2,5,300), 
                                np.array(["Lol that sentence is awesome for do1."]) ] ]
    return testing_data

In [4]:
test_temp = dummy_loadTestData()

In [5]:
test_temp[1]

[array(['Lol that sentence is awesome for do1.',
        'No way, this is irrelevent'], dtype='<U37'),
 array([[[0.13952658, 0.57798297, 0.71356846, ..., 0.09727206,
          0.08339716, 0.42132056],
         [0.97713175, 0.20028658, 0.27412616, ..., 0.19625067,
          0.3273923 , 0.0762757 ],
         [0.22154281, 0.07019896, 0.2722557 , ..., 0.42949362,
          0.18148447, 0.3250534 ],
         [0.90920583, 0.17384241, 0.1027518 , ..., 0.00801292,
          0.61239681, 0.50359767],
         [0.69925901, 0.66450187, 0.43771454, ..., 0.28400876,
          0.31714473, 0.79993739]],
 
        [[0.71283644, 0.42956767, 0.09803115, ..., 0.78058513,
          0.84218254, 0.42831162],
         [0.26065473, 0.38980305, 0.82476535, ..., 0.53272599,
          0.83900447, 0.93285412],
         [0.18889732, 0.43206335, 0.10429396, ..., 0.26024677,
          0.09137992, 0.29904015],
         [0.22707177, 0.84977624, 0.94196255, ..., 0.35684511,
          0.31490146, 0.89682339],
         [0.

In [26]:
def evaluate(model, testing_data, batch_size = 128, upper_bound = 100, threshold = 1):
    """
        Build the actual summaries for test data and evaluate them
        To do: 
            - load the actual x_test (embed test sentences) and y_test (compute rouge score)
        
        Parameters: 
            testing_data           - np.array 
                                        ex: [ doc1, doc2, ... , docn]
                                         where doci = [sentences, x_test, summary]
                                             where sentences = np.array of string
                                                   x_test = np.array of matrices (embedded sentences)
                                                   summaries = np.array of sentences
        
        Returns: 
            Rouge evaluations
    """   
    #rouge = Rouge()
    rouge = PyRouge(rouge_n=(1, 2, 4), rouge_l=True, rouge_w=True,
                rouge_w_weight=1.2, rouge_s=True, rouge_su=True, skip_gap=4)
    r1evals = []
    r2evals = []
    summaries = []    

    all_predicted_summary = []
    all_true_summary = []
    
    
    for doc in testing_data: 
        sentences = doc[0]
        
        x_test_old = doc[1]
        s1 = x_test_old.shape[0]
        (s3,s4) = x_test_old[0].shape
        print(s1,s3,s4)
        x_test = np.random.rand(s1,1,190,s4)
        for i in range(s1) :
            x_test[i] = np.array( [ np.pad(x_test_old[i], ((190-s3,0),(0,0)), 'constant') ] )
            

        true_summary = doc[2]
        
        predicted_scores = model.predict(x_test, batch_size=batch_size)
        #argsorted_scores= np.argsort(predicted_scores)
        argsorted_scores = np.argpartition(np.transpose(predicted_scores)[0], 1)
        
        predicted_summary = []
        summary_length = 0
        
        i = 0
        
        while i < len(sentences) and summary_length < upper_bound: 
            sentence = sentences[argsorted_scores[i]]
            #if ( dummy_rouge( sentence , predicted_summary ) < threshold ):
            sentence = np.array([sentence])
            #print(sentence, predicted_summary)
            predicted_summary.append(sentence)
            summary_length += len(nltk.word_tokenize(sentence[0]))
                
            i+=1
            
        #evals.append(dummy_rouge( predicted_summary, true_summary, alpha = N))
        #r1score = rouge.saliency(predicted_summary, true_summary, alpha=1)
        #r2score = rouge.saliency(predicted_summary, true_summary, alpha=0)

        temp = []
        for s in predicted_summary:
            temp.append(s[0])
        predicted_summary = '\n'.join(temp)
        
        for s in true_summary:
            temp.append(s)
        #print("**********************")
        #print(predicted_summary)
        #print(true_summary)
        #print("**********************")
        
        all_predicted_summary.append(predicted_summary)
        all_true_summary.append(true_summary)

        #evals.append(rouge.saliency(predicted_summary, true_summary, alpha=N))
        summaries.append((predicted_summary, true_summary))


    scores = rouge.evaluate_tokenized(all_predicted_summary, all_true_summary)
    print(" *--*--*--*--*--*--*--*")

    return scores
    

In [50]:
model = load_model('../models/model-nfilt-200.h5')
#joblib.dump(model, 'model_v1.pkl')
#model = joblib.load('model_v1.pkl')
#testing_data = dummy_loadTestData()
#testing_data = loadTestData("../data/DUC2002_Summarization_Documents")
#testing_data = loadTestData("../data/test_subset")
testing_data = loadTestData("../data/test_smallerset")
#testing_data = loadTestData("../data/subset/data/training/d01a")

In [121]:
rouge_scores = evaluate(model, testing_data[0:1], upper_bound=100,)
pprint(rouge_scores)

16 57 300
 *--*--*--*--*--*--*--*
{'rouge-1': {'f': 0.26691042047531993,
             'p': 0.15474297827239003,
             'r': 0.9700996677740864},
 'rouge-2': {'f': 0.23235563703024747,
             'p': 0.13455414012738853,
             'r': 0.8506711409395973},
 'rouge-4': {'f': 0.11244239631336407,
             'p': 0.06496272630457935,
             'r': 0.4178082191780822},
 'rouge-l': {'f': 0.26691042047531993,
             'p': 0.15474297827239003,
             'r': 0.9700996677740864},
 'rouge-s4': {'f': 0.23944700460829496,
              'p': 0.13833865814696486,
              'r': 0.8897260273972603},
 'rouge-su4': {'f': 0.24370779619398406,
               'p': 0.14085506475075393,
               'r': 0.9032992036405005},
 'rouge-w-1.2': {'f': 0.12431536423599997,
                 'p': 0.07214961508524717,
                 'r': 0.4488273544373458}}


In [70]:
len(testing_data)

6

In [43]:
print((testing_data[2][1][0]).shape)
v = 0
mx = 0
for st in testing_data[2][0]:
    #print(st)
    #o = input()    
    mx = max(mx, len(st.split()))
    v += len(st.split())
print(mx)


(38, 300)
35


## Code for Article extraction

In [158]:
def createSentenceEmbeddings(sentences):
    size = len(sentences)

    test_data = []
    count = 0
    max_size = 0
    
    documents_over_190 = 0
    sentences_over_190 = 0
    sentences_removed = 0
    over_190 = False

    arr = np.ones((len(sentences), 3), dtype=object) 
    arr[:,0] = "dummy"
    arr[:,1] = np.array(sentences)
    embedding = embed_sentences(arr)
    embedding = embedding[0::2]

    for e in embedding:
        if len(e) > max_size:
            max_size = len(e)
        if len(e) > 190:
            sentences_over_190 += 1
            over_190 = True
    if over_190:
        documents_over_190 += 1
        over_190 = False
        count -= len(sentences)
        sentences_removed += len(sentences)
        return

    count += len(sentences)
    test_data.append((np.array(sentences), np.array(embedding)))
    print("Finished", count, "of", size,"sentences --", count/size,"%", end='\r')

    return test_data

In [159]:
def preprocessing(sentences):
    # remove punctuations, numbers and special characters
    clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

    # make alphabets lowercase
    clean_sentences = [s.lower() for s in clean_sentences]
    return clean_sentences

In [166]:
def predict(model, data, batch_size = 128, upper_bound = 100, threshold = 1):
    """
        Predict the summary
        
        Parameters: 
            data           - np.array 
                                        ex: [ doc1, doc2, ... , docn]
                                         where doci = [sentences, x_test]
                                             where sentences = np.array of string
                                                   x_test = np.array of matrices (embedded sentences)        
        Returns: 
            Summary
    """   
    
    for doc in data: 
        sentences = doc[0]
        
        x_test_old = doc[1]
        s1 = x_test_old.shape[0]
        (s3,s4) = x_test_old[0].shape
        print(s1,s3,s4)
        x_test = np.random.rand(s1,1,190,s4)
        for i in range(s1) :
            x_test[i] = np.array( [ np.pad(x_test_old[i], ((190-s3,0),(0,0)), 'constant') ] )
        
        predicted_scores = model.predict(x_test, batch_size=batch_size)
        #argsorted_scores= np.argsort(predicted_scores)
        argsorted_scores = np.argpartition(np.transpose(predicted_scores)[0], 1)
        
        predicted_summary = []
        summary_length = 0
        
        i = 0
        
        while i < len(sentences) and summary_length < upper_bound: 
            sentence = sentences[argsorted_scores[i]]
            #if ( dummy_rouge( sentence , predicted_summary ) < threshold ):
            sentence = np.array([sentence])
            #print(sentence, predicted_summary)
            predicted_summary.append(sentence)
            summary_length += len(nltk.word_tokenize(sentence[0]))
                
            i+=1

        temp = []
        for s in predicted_summary:
            temp.append(s[0])
        predicted_summary = ' '.join(temp)
        return predicted_summary

    return []
    

In [167]:
def create_summary(model, article, summary_length):
    sentences = [sent_tokenize(article)]
    sentences = [y for x in sentences for y in x]
    clean_sentences = preprocessing(sentences)
    #print(clean_sentences)
    sentences_vector = createSentenceEmbeddings(clean_sentences)
#     print(sentences_vector[0][1].shape)
    return predict(model, sentences_vector, upper_bound=summary_length)


In [171]:
def summarize(model, url, summary_length):
    """
        :param summary_length: length of the summary in percentage
        :param url: url to scrape from the web
        :return: call the summarize function
        """
    # url = url.strip("https://")
    article_huff = Article(url)
    slept = 0
    article_huff.download()
    while article_huff.download_state == ArticleDownloadState.NOT_STARTED:
        # Raise exception if article download state does not change after 12 seconds
        if slept > 13:
            raise ArticleException('Download never started')
        sleep(1)
        slept += 1

    article_huff.parse()
    news_text, news_title = article_huff.text, article_huff.title
    summary = create_summary(model, article_huff.text, summary_length)
    return {"title": article_huff.title, "summary": summary, "article": news_text}

In [172]:
pprint(summarize(model, 'https://www.huffpost.com/entry/marco-rubio-donald-trump_n_607e5df5e4b063a636fb3f39', 100))

7 59 300 7 of 7 sentences -- 1.0 %
{'article': 'A conservative group is calling out Sen. Marco Rubio (R-Fla.) for '
            'hypocrisy when it comes to former President Donald Trump.\n'
            '\n'
            'Rubio warned in 2016 that Trump “would shatter the party and the '
            'conservative movement” and called him a “con artist.” But as the '
            'new video from the Republican Accountability Project points out, '
            'he’s not only turned into a Trump defender, he’s also won the '
            'former president’s endorsement for next year’s reelection '
            'effort.\n'
            '\n'
            '“This is an extra-special endorsement because he’s a resident and '
            'voter in Florida,” Rubio crowed last week. “So, I don’t just need '
            'his endorsement, I need his vote and the votes of all his family '
            'who are also moving to Florida.”\n'
            '\n'
            'The new video slams Rubio for the flip-fl

In [8]:
!curl https://raw.githubusercontent.com/prashanth018/Text-Summarization/shreya/BBCDataset.csv?token=ADB4OAHOA7VQTFA6QU5YIVDASQTWK -o BBCDataset.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7169k  100 7169k    0     0  13.1M      0 --:--:-- --:--:-- --:--:-- 13.2M


In [3]:
import pandas as pd

In [19]:
df = pd.read_csv('BBCDataset.csv', error_bad_lines=False)
df = df.iloc[:,0:2].copy()
df

,story,Summary
0,Holmes starts 2005 with GB events\n\nKelly Hol...,Holmes will make her first track appearance on...
1,Thanou desperate to make return\n\nGreek sprin...,"Thanou, 30, was provisionally suspended for mi..."
2,Cole faces lengthy injury lay-off\n\nAston Vil...,Aston Villa's Carlton Cole could be out for si...
3,Relay squad thrilled with honours\n\nJason Gar...,"""I think this award reinforces what we did on ..."
4,Hewitt survives Nalbandian epic\n\nHome favour...,Ninth seed Nalbandian had never come back from...
...,...,...
2220,Games maker fights for survival\n\nOne of Brit...,The administrators told BBC News Online that s...
2221,Mobile games come of age\n\nThe BBC News websi...,Even before Nokia's N-Gage game phone launched...
2222,Software watching while you work\n\nSoftware t...,The storage system has been incorporated into ...
2223,Warning over tsunami aid website\n\nNet users ...,She said the spam e-mails directing people to ...


In [22]:
df.iloc[0,0]

'Holmes starts 2005 with GB events\n\nKelly Holmes will start 2005 with a series of races in Britain.\n\nHolmes will make her first track appearance on home soil since winning double Olympic gold in January\'s Norwich Union International in Glasgow. She will also run in the Grand Prix in Birmingham in February and may defend her indoor AAA 800m title in Sheffield earlier that month. "I am still competitive and still want to win," she said. "I\'m an athlete and I can\'t wait to get back on the track." She added: "These events are also a great opportunity to thank the British public for the enormous levels of support they have given me from the moment I stepped off that plane from Greece." The Glasgow meeting will see Holmes compete over 1500m in a five-way match against Sweden, France, Russia and Italy.\n'

In [28]:
pd.DataFrame({
 'rouge-1': {'f': 0.26691042047531993,
             'p': 0.15474297827239003,
             'r': 0.9700996677740864},
 'rouge-2': {'f': 0.23235563703024747,
             'p': 0.13455414012738853,
             'r': 0.8506711409395973},
 'rouge-4': {'f': 0.11244239631336407,
             'p': 0.06496272630457935,
             'r': 0.4178082191780822},
 'rouge-l': {'f': 0.26691042047531993,
             'p': 0.15474297827239003,
             'r': 0.9700996677740864},
 'rouge-s4': {'f': 0.23944700460829496,
              'p': 0.13833865814696486,
              'r': 0.8897260273972603},
 'rouge-su4': {'f': 0.24370779619398406,
               'p': 0.14085506475075393,
               'r': 0.9032992036405005},
 'rouge-w-1.2': {'f': 0.12431536423599997,
                 'p': 0.07214961508524717,
                 'r': 0.4488273544373458}})

,rouge-1,rouge-2,rouge-4,rouge-l,rouge-s4,rouge-su4,rouge-w-1.2
f,0.266910,0.232356,0.112442,0.266910,0.239447,0.243708,0.124315
p,0.154743,0.134554,0.064963,0.154743,0.138339,0.140855,0.072150
r,0.970100,0.850671,0.417808,0.970100,0.889726,0.903299,0.448827


In [29]:
d = {'rouge-1': {'f': 0.26691042047531993,
             'p': 0.15474297827239003,
             'r': 0.9700996677740864},
 'rouge-2': {'f': 0.23235563703024747,
             'p': 0.13455414012738853,
             'r': 0.8506711409395973},
 'rouge-4': {'f': 0.11244239631336407,
             'p': 0.06496272630457935,
             'r': 0.4178082191780822},
 'rouge-l': {'f': 0.26691042047531993,
             'p': 0.15474297827239003,
             'r': 0.9700996677740864},
 'rouge-s4': {'f': 0.23944700460829496,
              'p': 0.13833865814696486,
              'r': 0.8897260273972603},
 'rouge-su4': {'f': 0.24370779619398406,
               'p': 0.14085506475075393,
               'r': 0.9032992036405005},
 'rouge-w-1.2': {'f': 0.12431536423599997,
                 'p': 0.07214961508524717,
                 'r': 0.4488273544373458}}

In [52]:
s = 'rouge-su4'
d[s]['p'] = 0.09127345541315364
d[s]['r'] = 0.89700996342470046
d[s]['f'] = (2 * d[s]['p'] * d[s]['r'])/ (d[s]['r'] + d[s]['p'])

In [54]:
pd.DataFrame(d)

,rouge-1,rouge-2,rouge-4,rouge-l,rouge-s4,rouge-su4,rouge-w-1.2
f,0.221245,0.223420,0.067229,0.221245,0.221409,0.165688,0.158784
p,0.125676,0.126305,0.036427,0.125676,0.126305,0.091273,0.087300
r,0.923562,0.966777,0.435356,0.923562,0.896313,0.897010,0.876474


In [55]:
d

{'rouge-1': {'f': 0.22124490379359626,
  'p': 0.12567562734523302,
  'r': 0.9235623465774087},
 'rouge-2': {'f': 0.2234203986939485,
  'p': 0.12630457099667733,
  'r': 0.9667774086567087},
 'rouge-4': {'f': 0.06722928904531722,
  'p': 0.03642726304577343,
  'r': 0.4353556972457827},
 'rouge-l': {'f': 0.22124490379359626,
  'p': 0.12567562734523302,
  'r': 0.9235623465774087},
 'rouge-s4': {'f': 0.22140913264046908,
  'p': 0.12630457099667733,
  'r': 0.8963133695443735},
 'rouge-su4': {'f': 0.16568769108374923,
  'p': 0.09127345541315364,
  'r': 0.8970099634247004},
 'rouge-w-1.2': {'f': 0.1587839459171986,
  'p': 0.08729966996676099,
  'r': 0.876474279444726}}